In [15]:
!pip install xgboost

  Using cached xgboost-2.1.1-py3-none-win_amd64.whl.metadata (2.1 kB)
Using cached xgboost-2.1.1-py3-none-win_amd64.whl (124.9 MB)


In [16]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import FinanceDataReader as fdr
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

In [17]:
kos_df = fdr.StockListing('KOSPI')
print(kos_df[kos_df['Name'] == '한화오션'][['Code']].iloc[0,0])
print(kos_df[kos_df['Name'] == '현대모비스'][['Code']].iloc[0,0])

042660
012330


In [18]:
HHO_df = fdr.DataReader('042660')
cars_df = fdr.DataReader('012330')

In [31]:
print(HHO_df.shape)
print(cars_df.shape)
print(HHO_df.isnull().sum())
print(cars_df.isnull().sum())

(5819, 7)
(5819, 7)
Date      0
Open      0
High      0
Low       0
Close     0
Volume    0
Change    1
dtype: int64
Date      0
Open      0
High      0
Low       0
Close     0
Volume    0
Change    1
dtype: int64


In [32]:
HHO_df

,Date,Open,High,Low,Close,Volume,Change
0,2001-02-02,1553,1787,1553,1787,9723154,NaN
1,2001-02-05,1860,1864,1562,1776,7686628,-0.006156
2,2001-02-06,2042,2042,1869,1994,7714656,0.122748
3,2001-02-07,1975,1975,1802,1807,3855729,-0.093781
4,2001-02-08,1687,1812,1687,1719,4004511,-0.048700
...,...,...,...,...,...,...,...
5814,2024-08-13,31350,32100,30900,30950,729398,-0.017460
5815,2024-08-14,31250,31300,30500,31250,793244,0.009693
5816,2024-08-16,31600,32050,31050,31800,1253041,0.017600
5817,2024-08-19,32000,32550,31200,31300,1131395,-0.015723


In [33]:
HHO_df = HHO_df.reset_index()
HHO_df

,index,Date,Open,High,Low,Close,Volume,Change
0,0,2001-02-02,1553,1787,1553,1787,9723154,NaN
1,1,2001-02-05,1860,1864,1562,1776,7686628,-0.006156
2,2,2001-02-06,2042,2042,1869,1994,7714656,0.122748
3,3,2001-02-07,1975,1975,1802,1807,3855729,-0.093781
4,4,2001-02-08,1687,1812,1687,1719,4004511,-0.048700
...,...,...,...,...,...,...,...,...
5814,5814,2024-08-13,31350,32100,30900,30950,729398,-0.017460
5815,5815,2024-08-14,31250,31300,30500,31250,793244,0.009693
5816,5816,2024-08-16,31600,32050,31050,31800,1253041,0.017600
5817,5817,2024-08-19,32000,32550,31200,31300,1131395,-0.015723


In [34]:
HHO_df_train = HHO_df[HHO_df['Date'] <= '2023-08-19']
HHO_df_test = HHO_df[HHO_df['Date'] > '2023-08-19']

In [41]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
n_days = 5
train_features = []
train_target = []

test_features = []
test_target = []
for i in range(n_days, len(HHO_df_train)):
    train_features.append(HHO_df_train[['Open', 'High', 'Low', 'Volume', 'Change']].iloc[i-n_days:i].values.flatten())
    train_target.append(HHO_df_train['Close'].iloc[i])

X_train = np.array(train_features)
y_train = np.array(train_target)

for i in range(n_days, len(HHO_df_test)):
    test_features.append(HHO_df_test[['Open', 'High', 'Low', 'Volume', 'Change']].iloc[i-n_days:i].values.flatten())
    test_target.append(HHO_df_test['Close'].iloc[i])

X_test = np.array(test_features)
y_test = np.array(test_target)

from sklearn.preprocessing import StandardScaler, RobustScaler
scaler = RobustScaler()


X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

model_HHO = RandomForestRegressor(n_estimators=200, random_state=0)
model_HHO.fit(X_train_scaled, y_train)

y_pred = model_HHO.predict(X_test_scaled)

rmse = mean_squared_error(y_test, y_pred) ** 0.5
score = model_HHO.score(X_test_scaled, y_test)
print(f'mse : {rmse}', f'R2 : {score}')

mse : 1096.7938866451088 R2 : 0.9085320987568392
